# Tutorial: Medical Record Analysis and Discharge Summary Generation

This tutorial demonstrates how to generate discharge summaries by analyzing and summarizing initial admission notes and progress notes during hospitalization.

# Step 1: Import Required Libraries and Load Data

First, we'll import the necessary libraries and load our data.

In [ ]:
# 리눅스 패키지 업데이트 및 cairo 설치
!apt update -y
!apt install -y libcairo2-dev pkg-config
!pip install pycairo

In [ ]:
!pip install -r requirements.txt

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

# Set cache directory for Hugging Face datasets and Transformers library
os.environ['HF_DATASETS_CACHE'] = '/home/jovyan/work/model/mllm_demo/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = '/home/jovyan/work/model/mllm_demo/transformers_cache'

In [2]:
# !pip install nltk

In [3]:
# !pip3 install torch torchvision torchaudio

In [4]:
import torch
print(torch.version)
print(torch.version.cuda)
print(torch.cuda.is_available())

<module 'torch.version' from '/usr/local/lib/python3.11/dist-packages/torch/version.py'>
12.4
True


In [5]:
# !pip install evaluate numpy pandas rouge transformers typing openpyxl accelerate

In [6]:
import json
import re
import torch
import evaluate
import numpy as np
import pandas as pd
from rouge import Rouge
from datetime import datetime
from typing import Dict, List, Tuple, Any
from transformers import MllamaForConditionalGeneration, AutoProcessor

# Load Data from Excel File
file_path = 'combined_patient_records_summary_demo.xlsx'
df = pd.read_excel(file_path)

2024-10-30 09:48:31.254371: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 09:48:31.272930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 09:48:31.295964: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 09:48:31.302990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 09:48:31.320223: I tensorflow/core/platform/cpu_feature_guar

# Step 2: Define Data Processing Functions

We'll define several functions to process and structure our data.

In [7]:
COLUMN_NAMES = {
    'item': '서식항목명',     # item name - Defines the type or category of the medical form element
    'content': '서식내용',    # content - Contains the actual medical information or notes
    'time': '서식작성일',     # creation date - When the medical record was created
    'category': '서식구분명',  # category name - Classifies the type of medical record
    'patient_id': '연구별 환자 ID'  # Patient ID for research - Unique identifier for each patient in the study
}

NOTE_TYPES = {
    'Admission_Note': '입원초진',        # Admission Note - First detailed examination upon hospital admission
    'Discharge_Summary': '퇴원기록',      # Discharge Summary - Final summary of the patient's hospital stay
    'Progress_Notes' : '입원경과'
}

def get_medical_note_sections() -> List[str]:
    """
    Returns the standard order of medical note sections.
    
    Returns:
        List[str]: Ordered list of standard medical note section names
    """
    return [
        "Present Illness", "Past History", "Social History", "Family History", "*History",
        "Review Of System", "Physical Examination", "소견", "Assessment", "Plan"
    ]

def get_section_priority(section_name: str) -> int:
    """
    Determines the priority order of medical note sections.
    
    Args:
        section_name: Name of the medical note section
        
    Returns:
        int: Priority index of the section (lower number means higher priority)
    """
    section_order = get_medical_note_sections()
    
    for index, pattern in enumerate(section_order):
        if pattern == "*History" and "History" in section_name and section_name not in section_order:
            return index
        if pattern in section_name:
            return index
    return len(section_order)

def convert_form_key_direction(form_key: str) -> str:
    """
    Converts arrow direction in form item names from '<-' to '->'.
    
    Args:
        form_key: Original form key with '<-' direction
        
    Returns:
        str: Converted form key with '->' direction
    """
    return ' -> '.join(reversed(form_key.split(' <- ')))

def parse_medical_note(df: Any) -> Tuple[Dict[str, str], Any]:
    """
    Extracts and organizes medical note content from DataFrame.
    
    Args:
        df: DataFrame containing medical note data
        
    Returns:
        Tuple[Dict[str, str], Any]: A tuple containing:
            - Dictionary of sorted medical note content
            - Timestamp of the note
    """
    medical_note_content = {}
    note_timestamp = None

    for _, row in df.iterrows():
        form_item = convert_form_key_direction(row[COLUMN_NAMES['item']])
        content = row[COLUMN_NAMES['content']]
        medical_note_content[form_item] = content

        if note_timestamp is None:
            note_timestamp = row[COLUMN_NAMES['time']]

    return dict(sorted(medical_note_content.items(), key=lambda x: get_section_priority(x[0]))), note_timestamp

def format_discharge_note(note_content: Dict[str, str]) -> str:
    """
    Formats discharge note content with proper indentation and structure.
    
    Args:
        note_content: Dictionary containing discharge note content
        
    Returns:
        str: Formatted discharge note with proper indentation
    """
    formatted_lines = []
    for section_key, content in note_content.items():
        section_hierarchy = section_key.split(' -> ')
        for level, section in enumerate(section_hierarchy):
            formatted_lines.append('    ' * level + section)
        
        cleaned_content = content.replace('_x000D_\n', '\n').replace('*x000D*\n', '\n')
        content_lines = cleaned_content.split('\n')
        indented_content = '\n'.join('    ' * len(section_hierarchy) + line.strip() 
                                    for line in content_lines if line.strip())
        formatted_lines.append(indented_content)
        formatted_lines.append('')
    
    return '\n'.join(formatted_lines)

def format_note_date(timestamp: Any) -> str:
    """
    Formats medical note timestamp into standardized date string.
    
    Args:
        timestamp: Date timestamp in various possible formats
        
    Returns:
        str: Standardized date string in 'YYYY-MM-DD' format
    """
    try:
        date_obj = datetime.strptime(str(timestamp), '%Y-%m-%d')
        return date_obj.strftime('%Y-%m-%d')
    except ValueError:
        return str(timestamp)

def generate_formatted_notes(df: Any, note_type: str, is_discharge_note: bool = False) -> str:
    """
    Generates formatted medical notes from DataFrame for specified note type.
    
    Args:
        df: DataFrame containing medical records
        note_type: Type of medical note ('Admission_Note' or 'Discharge_Summary')
        is_discharge_note: Boolean flag for discharge note formatting
        
    Returns:
        str: Formatted medical notes as a single string
    """
    formatted_notes = []
    category = NOTE_TYPES[note_type]
    
    category_df = df[df[COLUMN_NAMES['category']] == category]
    if not category_df.empty:
        note_content, timestamp = parse_medical_note(category_df)
        
        if is_discharge_note:
            discharge_patterns = r'입원사유.*병력요약|입원경과'  # Reason for admission.*History summary|progress notes
            note_content = {k: v for k, v in note_content.items() 
                          if re.search(discharge_patterns, k)}
            if note_content:
                formatted_text = format_discharge_note(note_content)
                formatted_notes.extend([
                    f">>>{category}<<<",
                    format_note_date(timestamp),
                    formatted_text
                ])
        else:
            formatted_text = json.dumps(note_content, ensure_ascii=False, indent=4)
            formatted_notes.extend([
                f">>>{category}<<<",
                format_note_date(timestamp),
                formatted_text
            ])

    return "\n".join(formatted_notes)

def process_patient_records(df: Any) -> Dict[Any, Dict[str, str]]:
    """
    Processes and organizes medical records for multiple patients.
    
    Args:
        df: DataFrame containing all patient medical records
        
    Returns:
        Dict[Any, Dict[str, str]]: Dictionary containing processed records for each patient:
            - Key: Patient ID
            - Value: Dictionary with 'Admission_Note' and 'Discharge_Summary' keys
    """
    patient_records = {}
    
    for patient_id in df[COLUMN_NAMES['patient_id']].unique():
        patient_df = df[df[COLUMN_NAMES['patient_id']] == patient_id]
        
        patient_records[patient_id] = {
            'Admission_Note': generate_formatted_notes(patient_df, 'Admission_Note'),
            'Discharge_Summary': generate_formatted_notes(patient_df, 'Discharge_Summary', 
                                                       is_discharge_note=True),
            'Progress_Notes' : generate_formatted_notes(patient_df, 'Progress_Notes')
        }
    
    return patient_records

In [ ]:
# # Check Patient Record in Korean
patient_records = process_patient_records(df)

# Step 3: Load Model

Load the Llama model for analyzing and summarizing medical records.

In [9]:
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
token = ""

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    temperature = 0.1,
    token = token
)
processor = AutoProcessor.from_pretrained(model_id)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:06<00:00,  1.31s/it]
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 30] Read-only file system: '/home/jovyan/work/model/mllm_demo/transformers_cache/models--meta-llama--Llama-3.2-11B-Vision-Instruct/.no_exist/cee5b78e6faed15d5f2e6d8a654fd5b247c0d5ca/processor_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 30] Read-only file system: '/home/jovyan/work/model/mllm_demo/transformers_cache/models--meta-llama--Llama-3.2-11B-Vision-Instruct/.no_exist/cee5b78e6faed15d5f2e6d8a654fd5b247c0d5ca/processor_config.json'


## 3-1. Define Section Extraction Functions

Define functions to extract specific sections from the text.

In [10]:
def extract_discharge_summary_sections(discharge_text: str) -> str:
    """Extracts specific sections from patient's discharge summary."""
    target_sections = ['입원경과', '입원사유 및 병력요약']  # 'Hospital Course', 'Reason for Admission and Summary of Medical History'
    text_lines = discharge_text.split('\n')
    formatted_sections = []
    section_start_indices = []

    for line_index in reversed(range(len(text_lines))):
        if text_lines[line_index].strip() in target_sections:
            section_start_indices.append(line_index)
            if len(section_start_indices) == len(target_sections):
                break

    if len(section_start_indices) < len(target_sections):
        return discharge_text.strip()

    current_line_index = section_start_indices[1]

    while current_line_index < len(text_lines):
        current_line = text_lines[current_line_index].rstrip()
        
        if current_line.strip() in target_sections:
            formatted_sections.append(current_line)
            current_line_index += 1
            
            while current_line_index < len(text_lines):
                next_line = text_lines[current_line_index].rstrip()
                
                if next_line.startswith(('    ', '\t')):
                    formatted_sections.append(next_line)
                    current_line_index += 1
                elif next_line.strip() in target_sections:
                    break
                else:
                    current_line_index += 1
                    
            formatted_sections.append("")
        else:
            current_line_index += 1

    return '\n'.join(formatted_sections).strip()

## 3-2. Generate Medical Record Summaries Using the Model

### Summarize each patient's medical records using the loaded model.

This example shows a medical record structure where both input (Initial Admission Note, Progress Note) and output (Discharge Summary) were translated from Korean to English for better understanding. 

In [11]:
def create_llm_prompt(admission_notes: str) -> List[Dict[str, str]]:
    """
    Creates a prompt for the LLM to generate discharge summary.
    
    Args:
        admission_notes: Cleaned admission and progress notes
    
    Returns:
        List of message dictionaries for the LLM
    """
    system_prompt = """
    You are an attending physician (specialist) collaborating with a resident doctor (house staff) to write a discharge summary based on patient records. Your goal is to provide accurate and complete information without adding any assumptions or interpretations not present in the original data.
    """
    
    user_prompt = f"""
    Using the patient data provided below, focus on extracting and presenting key information for '입원경과' (Hospital Course) and '입원사유 및 병력요약' (Reason for Admission and Summary of Medical History). Use the information from the '입원초진' (Admission Initial Assessment) and '입원경과' (Hospital Course) sections.
    
    Patient Data: {admission_notes}
    
    Follow the output format below:
    입원경과
        [Admission Date] Adm
        [Operation Date] Op

    입원사유 및 병력요약
        Description
            [Content of the "Present Illness" field from '입원초진']
    
    Guidelines:
    - **Admission Date**: Use the date that appears immediately after '>>>입원초진<<<'.
    - **Operation Date**: Use the date that appears immediately after '>>>입원경과<<<'.
    - **Description**: Use the content from the "Present Illness" field in the '입원초진' section. **If the "Present Illness" field is brief or lacks detail, include additional relevant information from the '입원초진' and '입원경과' section to provide a complete summary.**
    - **Format**: Follow the format and style provided above exactly, including indentation and line breaks.
    - **Content**: Use only the information explicitly included in the provided patient data. Do not include interpretations, assumptions, or additional information not present in the data.
    - **Consistency**: Apply this format consistently across various cases and adjust the output according to the provided data.
    - **Prevent Hallucinations**: Do not include information that is not present in the data. Only extract and present the information provided.
    
    Generate the output according to the above guidelines.
    """
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

def generate_discharge_summary(model: Any, processor: Any, admission_notes: str) -> str:
    """
    Generates discharge summary using the LLM model.
    
    Args:
        model: The LLM model
        processor: Text processor for the model
        admission_notes: Patient's admission and progress notes
        
    Returns:
        Generated discharge summary text
    """
    messages = create_llm_prompt(admission_notes)
    
    model_input = processor.apply_chat_template(
        messages, 
        add_generation_prompt=True
    )
    
    encoded_input = processor(
        text=model_input,
        add_special_tokens=True,
        return_tensors="pt"
    ).to(model.device)
    
    generated_output = model.generate(
        **encoded_input, 
        max_new_tokens=800
    )
    
    full_text = processor.decode(
        generated_output[0], 
        skip_special_tokens=True
    )
    
    return extract_discharge_summary_sections(full_text)

def process_patient_discharge_summaries(
    model: Any,
    processor: Any,
    patient_records: Dict[str, Dict[str, str]]
) -> Tuple[List[str], List[str]]:
    """
    Processes discharge summaries for multiple patients.
    
    Args:
        model: The LLM model
        processor: Text processor for the model
        patient_records: Dictionary containing patient records
        
    Returns:
        Tuple containing lists of generated summaries and actual summaries
    """
    generated_summaries = []
    actual_summaries = []
    
    for index, (patient_id, records) in enumerate(patient_records.items()):
        print(f"Processing Patient #{index} =======")
        
        combined_notes = f"{records['Admission_Note']}\n{records['Progress_Notes']}"
        cleaned_notes = combined_notes.replace(r'_x000D_\n', '')
        
        generated_summary = generate_discharge_summary(
            model=model,
            processor=processor,
            admission_notes=cleaned_notes
        )
        
        generated_summaries.append(generated_summary)
        actual_summaries.append(records['Discharge_Summary'])
        
        print(generated_summary)
    
    return generated_summaries, actual_summaries

def translate_summary(model: Any, processor: Any, summary: str) -> str:
    """
    Translates the given Korean summary into English while maintaining the format.
    
    Args:
        model: The LLM model
        processor: Text processor for the model
        summary: The Korean summary text to translate
        
    Returns:
        Translated summary text in English
    """
    messages = [
        {
            "role": "system",
            "content": "You are a professional medical translator. Translate the following Korean medical summary into English, maintaining the exact format, indentation, and line breaks. Do not add or omit any information."
        },
        {
            "role": "user",
            "content": summary
        }
    ]
    
    model_input = processor.apply_chat_template(
        messages, 
        add_generation_prompt=True
    )
    
    encoded_input = processor(
        text=model_input,
        add_special_tokens=True,
        return_tensors="pt"
    ).to(model.device)
    
    generated_output = model.generate(
        **encoded_input, 
        max_new_tokens=800
    )
    
    translated_summary = processor.decode(
        generated_output[0], 
        skip_special_tokens=True
    )
    
    return translated_summary

def translate_summaries(
    model: Any,
    processor: Any,
    summaries: List[str]
) -> List[str]:
    """
    Translates a list of Korean summaries into English while maintaining their formats.
    
    Args:
        model: The LLM model
        processor: Text processor for the model
        summaries: List of Korean summaries to translate
        
    Returns:
        List of translated summaries in English
    """
    translated_summaries = []
    
    for index, summary in enumerate(summaries):
        print(f"Translating Summary #{index} =======")
        
        translated_summary = translate_summary(
            model=model,
            processor=processor,
            summary=summary
        )

        extracted_text = extract_reason_for_hospitalization_summary(translated_summary)
        translated_summaries.append(extracted_text)
        print(extracted_text)
    
    return translated_summaries

def extract_reason_for_hospitalization_summary(text):
    start_section = "assistant"
    
    if start_section in text:
        extracted_text = text.split(start_section, 1)[1]
        return extracted_text.strip()
    else:
        return "Section not found in the text."

# Main execution
generated_summaries, actual_summaries = process_patient_discharge_summaries(
    model=model,
    processor=processor,
    patient_records=patient_records
)

translated_summaries = translate_summaries(
    model=model,
    processor=processor,
    summaries=generated_summaries
)

Processing Patient #0 =======
입원경과
    2023-03-03 Adm
    2023-05-09 Op

입원사유 및 병력요약
    Description
        상환 양손 저린감으로 내원하여 CTS, B/L(Bland 1/3)에 대해 외래 추시중이던 분으로, 보존적 치료로 나아지지 않는 증상으로 금번 CTR, Lt.위해 내원.
Processing Patient #1 =======
입원경과
    2023-03-15 Adm
    2023-03-15 Op

입원사유 및 병력요약
    Description
        상환 양손 저림증상으로 내원하여 CTS, B/L에 대해 외래 추시중이던 분으로, 보존적 치료로 호전 보이지 않아 금번 CTR, Rt + A1 pulley release, 3th finger, Rt. 위해 내원.
Processing Patient #2 =======
입원경과
    2023-03-26 Adm
    2023-05-11 Op

입원사유 및 병력요약
    Description
        상환 2년전부터 시작된 Lt. 2nd finger의 triggering을 주소로 외래추시중이던 분으로, 보존적 치료에 반응하지 않아 금번 A1 pulley release, 2nd finger, Lt. 위해 내원.
Processing Patient #3 =======
입원경과
    2023-04-07 Adm
    2023-05-12 Op

입원사유 및 병력요약
    Description
        # DDLT recipient
Processing Patient #4 =======
입원경과
    2023-04-27 Adm
    2023-03 Op

입원사유 및 병력요약
    Description
        상환 3번쨰 손가락의 triggering으로 외래 방문하여, A1 pulley release 수술 위해 입원함.
Translating Summary #0 =======
Admission and Di

In [12]:
# Translating an actual Discharge Summary into English
discharge_summaries = []

for patient_id, records in patient_records.items():
    discharge_summary = records['Discharge_Summary'].replace(r'_x000D_\n', '')
    discharge_summaries.append(discharge_summary)

In [14]:
translated_actual_summaries = translate_summaries(
    model=model,
    processor=processor,
    summaries=discharge_summaries
)

Translating Summary #0 =======
>>>Discharge Record<<<
2023-05-14 00:00:00
Admission and Operation History
    2023-03 Admission
    2023-03 Operation

Reason for Hospitalization and Past Medical History Summary
    Description
        Admitted with a complaint of bilateral hand numbness and was a follow-up patient for CTS (Carpal Tunnel Syndrome) and B/L (Bilateral) (Bland 1/3). As the symptoms did not improve with conservative treatment, the patient was admitted for CTR (Carpal Tunnel Release) and Lt. (Left) surgery.
Translating Summary #1 =======
>>>Discharge Record<<<
2023-05-23 00:00:00
Admission History
    2023-03 Admitted
    2023-03 Operated

Reason for Hospitalization and Past Medical History Summary
    Description
        Admitted with symptoms of bilateral hand numbness and visited for follow-up for CTS (Carpal Tunnel Syndrome) and B/L (bilateral). As conservative treatment did not show improvement,
        This time, admitted for CTR (Carpal Tunnel Release) and Rt + A1 pul

### Compare Model Output with Actual Reference Data

In [70]:
patient_id = 0

print('<<<Model Output>>>\n')
print(translated_summaries[patient_id])
translated_actual_summaries
print('\n', '='*10, 'Comparison between Model Output and Reference Data', '='*10, '\n')

print('<<<Reference Data>>>\n')
print(translated_actual_summaries[patient_id])

<<<Model Output>>>

Admission and Discharge Information
    Admission Date: 2023-03-03
    Discharge Date: 2023-05-09

Reason for Hospitalization and Medical History Summary
    Description
        Admitted with symptoms of bilateral hand numbness and tingling, and was previously followed up by an outpatient doctor for CTS (Carpal Tunnel Syndrome) and B/L (Bilateral) (Bland 1/3). However, the symptoms did not improve with conservative treatment, and this time admitted for CTR (Carpal Tunnel Release) and Lt. (Left) wrist surgery.

 ========== Comparison between Model Output and Reference Data ========== 

<<<Reference Data>>>

>>>Discharge Record<<<
2023-05-14 00:00:00
Admission and Operation History
    2023-03 Admission
    2023-03 Operation

Reason for Hospitalization and Past Medical History Summary
    Description
        Admitted with a complaint of bilateral hand numbness and was a follow-up patient for CTS (Carpal Tunnel Syndrome) and B/L (Bilateral) (Bland 1/3). As the symptoms

# Step 4: Summary Quality Evaluation

Evaluate the quality of generated summaries using ROUGE-L and METEOR metrics.

In [15]:
rouge = Rouge()
meteor = evaluate.load('meteor')

results = {
    'ROUGE-L': [],
    'METEOR': []
}

for pred, ref in zip(translated_summaries, translated_actual_summaries):
    # Calculate ROUGE scores
    rouge_scores = rouge.get_scores(pred, ref)[0]
    results['ROUGE-L'].append(rouge_scores['rouge-l']['f'])
    
    # Calculate METEOR score
    meteor_score = meteor.compute(predictions=[pred], references=[ref])['meteor']
    results['METEOR'].append(meteor_score)

for metric, scores in results.items():
    avg_score = np.mean(scores)
    print(f"Average {metric} score: {avg_score:.4f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Average ROUGE-L score: 0.5780
Average METEOR score: 0.5237


In [16]:
# When executed in Korean

# Extract Relevant Sections for Comparison with Reference Data
new_label_list = []
for item in actual_summaries:
    if len(item) > 0 and isinstance(item[0], str): 
        new_label_list.append('\n'.join(item.split('\n')[2:]))

rouge = Rouge()
meteor = evaluate.load('meteor')

results = {
    'ROUGE-L': [],
    'METEOR': []
}

for pred, ref in zip(generated_summaries, new_label_list):
    # Calculate ROUGE scores
    rouge_scores = rouge.get_scores(pred, ref)[0]
    results['ROUGE-L'].append(rouge_scores['rouge-l']['f'])
    
    # Calculate METEOR score
    meteor_score = meteor.compute(predictions=[pred], references=[ref])['meteor']
    results['METEOR'].append(meteor_score)

for metric, scores in results.items():
    avg_score = np.mean(scores)
    print(f"Average {metric} score: {avg_score:.4f}")

Average ROUGE-L score: 0.6713
Average METEOR score: 0.6990


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
